In [517]:
import pandas as pd
train=pd.read_csv("C:/Users/sshyu/Downloads/Aimers_data/train.csv").drop(columns=['ID'])
test=pd.read_csv("C:/Users/sshyu/Downloads/Aimers_data/test.csv").drop(columns=['ID'])
submission=pd.read_csv("C:/Users/sshyu/Downloads/Aimers_data/sample_submission.csv")

## 데이터 확인 ##

In [518]:
train.shape[0]

256351

In [519]:
pd.DataFrame(train.isna().sum()[train.isna().sum() > 0])

,0
임신 시도 또는 마지막 임신 경과 연수,246981
특정 시술 유형,2
단일 배아 이식 여부,6291
착상 전 유전 검사 사용 여부,253633
착상 전 유전 진단 사용 여부,6291
배아 생성 주요 이유,6291
총 생성 배아 수,6291
미세주입된 난자 수,6291
미세주입에서 생성된 배아 수,6291
이식된 배아 수,6291


In [520]:
# 너무 NaN이 많은 컬럼 제외
train.drop(['임신 시도 또는 마지막 임신 경과 연수','착상 전 유전 검사 사용 여부','난자 해동 경과일','배아 해동 경과일'],axis=1,inplace=True)
test.drop(['임신 시도 또는 마지막 임신 경과 연수','착상 전 유전 검사 사용 여부','난자 해동 경과일','배아 해동 경과일'],axis=1,inplace=True)

In [521]:
# 수/ 여부 가 들어간 column 중 nan 값을 0으로 채움
train.loc[:, train.columns.str.contains("수|여부")] = train.loc[:, train.columns.str.contains("수|여부")].fillna(0)
test.loc[:, test.columns.str.contains("수|여부")] = test.loc[:, test.columns.str.contains("수|여부")].fillna(0)

In [522]:
pd.DataFrame(train.isna().sum()[train.isna().sum() > 0])

,0
특정 시술 유형,2
배아 생성 주요 이유,6291
난자 채취 경과일,57488
난자 혼합 경과일,53735
배아 이식 경과일,43566


In [523]:
pd.DataFrame(test.isna().sum()[test.isna().sum() > 0])

,0
배아 생성 주요 이유,2176
난자 채취 경과일,19949
난자 혼합 경과일,18579
배아 이식 경과일,15246


In [524]:
# train에서 특정 시술 유형이 NaN인 것 제외
train = train.dropna(subset=["특정 시술 유형"])

In [525]:
pd.DataFrame(train.isna().sum()[train.isna().sum() > 0])

,0
배아 생성 주요 이유,6291
난자 채취 경과일,57488
난자 혼합 경과일,53735
배아 이식 경과일,43564


In [526]:
# 난자 채취 경과일, 난자 혼합 경과일의 경우 csv 를 확인해보니 0 아니면 NaN 값임 -> 제외시키기
train.drop(['난자 채취 경과일','난자 혼합 경과일'],inplace=True, axis=1)
test.drop(['난자 채취 경과일','난자 혼합 경과일'],inplace=True, axis=1)

In [527]:
pd.DataFrame(train.isna().sum()[train.isna().sum() > 0])

,0
배아 생성 주요 이유,6291
배아 이식 경과일,43564


In [528]:
pd.DataFrame(test.isna().sum()[train.isna().sum() > 0])

,0
배아 생성 주요 이유,2176
배아 이식 경과일,15246


In [529]:
train['시술 유형'].value_counts()

시술 유형
IVF    250058
DI       6291
Name: count, dtype: int64

In [530]:
# 시술 유형이 IVF면 1, DI면 0으로
train.loc[train['시술 유형']=='IVF','시술 유형']=1
train.loc[train['시술 유형']=='DI','시술 유형']=0

In [531]:
test.loc[test['시술 유형']=='IVF','시술 유형']=1
test.loc[test['시술 유형']=='DI','시술 유형']=0

In [532]:
train['시술 유형']=train['시술 유형'].astype(int)
test['시술 유형']=test['시술 유형'].astype(int)

In [533]:
train.select_dtypes(include=['object']).columns

Index(['시술 시기 코드', '시술 당시 나이', '특정 시술 유형', '배란 유도 유형', '배아 생성 주요 이유',
       '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수',
       'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '난자 출처',
       '정자 출처', '난자 기증자 나이', '정자 기증자 나이'],
      dtype='object')

In [534]:
train['시술 시기 코드'].value_counts()

시술 시기 코드
TRDQAZ    38969
TRCMWS    38090
TRYBLT    36713
TRVNRY    36172
TRJXFG    36030
TRZKPL    35544
TRXQMD    34831
Name: count, dtype: int64

In [535]:
train.groupby('시술 시기 코드')['임신 성공 여부'].mean()

시술 시기 코드
TRCMWS    0.257417
TRDQAZ    0.245426
TRJXFG    0.266278
TRVNRY    0.259787
TRXQMD    0.256122
TRYBLT    0.269087
TRZKPL    0.255120
Name: 임신 성공 여부, dtype: float64

In [536]:
test['시술 시기 코드'].value_counts()

시술 시기 코드
TRDQAZ    13610
TRCMWS    13392
TRYBLT    12929
TRZKPL    12793
TRJXFG    12764
TRVNRY    12560
TRXQMD    12019
Name: count, dtype: int64

시술 시기 코드의 경우 적절히 분배되어 있고, 시술 시기 코드 별 임신 성공 여부에 큰 차이가 없음

따라서 해당 열을 제외

In [537]:
train.drop('시술 시기 코드',axis=1,inplace=True)
test.drop('시술 시기 코드',axis=1,inplace=True)

In [538]:
# 따라서 배란 유도 유형 drop -> 의미 있는 값이 2개 밖에 없음
train.drop('배란 유도 유형', inplace=True, axis=1)
test.drop('배란 유도 유형', inplace=True, axis=1)

In [539]:
train['특정 시술 유형'].value_counts()

특정 시술 유형
ICSI                                   122368
IVF                                     91755
Unknown                                 26939
IUI                                      6100
ICSI:ICSI                                2314
ICSI / BLASTOCYST                        1609
IVF / BLASTOCYST                         1248
IVF:IVF                                  1146
ICSI:IVF                                  873
ICSI / AH                                 769
IVF:ICSI                                  392
IVF / AH                                  319
ICSI:Unknown                              207
IVF:Unknown                               100
ICI                                        96
Generic DI                                 71
IVI                                        23
ICSI / BLASTOCYST:IVF / BLASTOCYST          6
ICSI / BLASTOCYST :IVF / BLASTOCYST         4
FER                                         3
IVF / AH:ICSI / AH                          2
GIFT                     

In [540]:
# 너무 카테고리가 많음
# 주요 그룹으로 단순화하기 - 카테고리 갯수 줄이기
def simplify_treatment_type(x):
    if "ICSI" in x:
        return "ICSI"
    elif "IVF" in x:
        return "IVF"
    elif "IUI" in x:
        return "IUI"
    elif "FER" in x:
        return "FER"
    elif "GIFT" in x:
        return "GIFT"
    elif "ICI" in x:
        return "ICI"
    elif "DI" in x:
        return "DI"
    else:
        return "Unknown"

train['특정 시술 유형'] = train['특정 시술 유형'].apply(simplify_treatment_type)

In [541]:
train['특정 시술 유형'].value_counts()

특정 시술 유형
ICSI       128547
IVF         94568
Unknown     26962
IUI          6100
ICI            96
DI             71
FER             3
GIFT            2
Name: count, dtype: int64

In [542]:
test['특정 시술 유형'] = test['특정 시술 유형'].apply(simplify_treatment_type)

In [543]:
test['특정 시술 유형'].value_counts()

특정 시술 유형
ICSI       45383
IVF        33292
Unknown     9224
IUI         2114
ICI           28
DI            26
Name: count, dtype: int64

In [544]:
train.groupby('특정 시술 유형')['임신 성공 여부'].mean()

특정 시술 유형
DI         0.169014
FER        0.333333
GIFT       0.000000
ICI        0.135417
ICSI       0.268314
IUI        0.128525
IVF        0.259443
Unknown    0.237074
Name: 임신 성공 여부, dtype: float64

In [545]:
# DI (기증 정자 주입)은 성공 여부가 0
# 하지만 test 데이터에는 해당 데이터가 없음
# 그럼 DI가 있는 row를 드롭할까?
train = train[train['특정 시술 유형'] != 'DI']

In [546]:
train['배아 생성 주요 이유'].value_counts()

배아 생성 주요 이유
현재 시술용                 233730
배아 저장용                   9192
기증용, 현재 시술용              3784
난자 저장용                   1959
기증용                      1108
기증용, 배아 저장용               125
배아 저장용, 현재 시술용             83
기증용, 난자 저장용                44
기증용, 배아 저장용, 현재 시술용        20
난자 저장용, 배아 저장용              6
난자 저장용, 현재 시술용              5
연구용, 현재 시술용                 1
난자 저장용, 배아 저장용, 연구용         1
Name: count, dtype: int64

In [547]:
test['배아 생성 주요 이유'].value_counts()

배아 생성 주요 이유
현재 시술용                 82253
배아 저장용                  3151
기증용, 현재 시술용             1327
난자 저장용                   664
기증용                      384
기증용, 배아 저장용               47
배아 저장용, 현재 시술용            38
기증용, 난자 저장용               19
기증용, 배아 저장용, 현재 시술용        6
난자 저장용, 현재 시술용             1
난자 저장용, 배아 저장용             1
Name: count, dtype: int64

In [548]:
train[train['배아 생성 주요 이유'].str.contains('현재 시술용',na=False)].shape

(237623, 60)

In [549]:
train.groupby('배아 생성 주요 이유')['임신 성공 여부'].mean()

배아 생성 주요 이유
기증용                    0.000000
기증용, 난자 저장용            0.000000
기증용, 배아 저장용            0.000000
기증용, 배아 저장용, 현재 시술용    0.350000
기증용, 현재 시술용            0.379757
난자 저장용                 0.000000
난자 저장용, 배아 저장용         0.000000
난자 저장용, 배아 저장용, 연구용    0.000000
난자 저장용, 현재 시술용         0.200000
배아 저장용                 0.000870
배아 저장용, 현재 시술용         0.265060
연구용, 현재 시술용            0.000000
현재 시술용                 0.273572
Name: 임신 성공 여부, dtype: float64

In [550]:
train[train['배아 생성 주요 이유'].isna()]['임신 성공 여부'].mean()

0.1284565916398714

현재 시술 용만 배아 생성 주요 이유 = 1 을 할지, 아니면 현재 시술 용이 포함된 걸 모두 배아 생성 주요 이유 = 0 을 할지 고민했는데 (아닌 건 0으로)

배아 생성 주요 이유에 현재 시술용이 포함되어 있는 것과 아닌 것에 임신 성공 여부 차이가 많이 남

또한 배아 생성 주요 이유가 NaN 값인 건 그 중간 정도 임신 성공 여부 평균값을 가짐

나중에 scaling을 한다고 했을 때 모든 값들이 0~1 사이, 또는 -1~1 사이의 값들이어야 하므로

배아 생성 주요 이유에 현재 시술용이 포함되면 1, 아니면 0, NaN이면 0.5 부여

In [551]:
train[train['배아 생성 주요 이유'].isna()].shape

(6220, 60)

In [552]:
train['현재 시술용 여부']=0.0
train.loc[train['배아 생성 주요 이유'].isna(),'현재 시술용 여부']=0.5

In [553]:
train['현재 시술용 여부'].value_counts()

현재 시술용 여부
0.0    250058
0.5      6220
Name: count, dtype: int64

In [554]:
train.loc[train['배아 생성 주요 이유'].str.contains('현재 시술용',na=False),'현재 시술용 여부']=1

In [555]:
train.drop('배아 생성 주요 이유',inplace=True,axis=1)
train['현재 시술용 여부'].value_counts()

현재 시술용 여부
1.0    237623
0.0     12435
0.5      6220
Name: count, dtype: int64

In [556]:
test['현재 시술용 여부']=0.0
test.loc[test['배아 생성 주요 이유'].isna(),'현재 시술용 여부']=0.5
test.loc[test['배아 생성 주요 이유'].str.contains('현재 시술용',na=False),'현재 시술용 여부']=1

In [557]:
test.drop('배아 생성 주요 이유',inplace=True,axis=1)
test['현재 시술용 여부'].value_counts()

현재 시술용 여부
1.0    83625
0.0     4266
0.5     2176
Name: count, dtype: int64

In [558]:
train.groupby('총 시술 횟수')['임신 성공 여부'].mean()

총 시술 횟수
0회       0.291018
1회       0.249969
2회       0.244190
3회       0.236400
4회       0.227765
5회       0.215753
6회 이상    0.203374
Name: 임신 성공 여부, dtype: float64

In [559]:
# 상관관계 분석을 위해 일단 숫자형으로 바꾸기
train['총 시술 횟수'] = train['총 시술 횟수'].str.extract('(\d+)').astype(int)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\sshyu\AppData\Local\Temp\ipykernel_12052\3497006103.py:2: SyntaxWarning: invalid escape sequence '\d'
  train['총 시술 횟수'] = train['총 시술 횟수'].str.extract('(\d+)').astype(int)


In [560]:
test['총 시술 횟수'] = test['총 시술 횟수'].str.extract('(\d+)').astype(int)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\sshyu\AppData\Local\Temp\ipykernel_12052\1609410155.py:1: SyntaxWarning: invalid escape sequence '\d'
  test['총 시술 횟수'] = test['총 시술 횟수'].str.extract('(\d+)').astype(int)


In [561]:
train[['총 시술 횟수', '임신 성공 여부']].corr()

,총 시술 횟수,임신 성공 여부
총 시술 횟수,1.000000,-0.058927
임신 성공 여부,-0.058927,1.000000


In [562]:
train[['총 시술 횟수', '임신 성공 여부']].corr(method='spearman')

,총 시술 횟수,임신 성공 여부
총 시술 횟수,1.000000,-0.062404
임신 성공 여부,-0.062404,1.000000


In [563]:
# 어느 정도 관계가 보이긴 함.. 냅두기
train.groupby('클리닉 내 총 시술 횟수')['임신 성공 여부'].mean()
# 마찬가지로 어느 정도 관계가 보이긴 함.. 냅두기 이때 숫자형으로 바꾸기
train['클리닉 내 총 시술 횟수'] = train['클리닉 내 총 시술 횟수'].str.extract('(\d+)').astype(int)

<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
C:\Users\sshyu\AppData\Local\Temp\ipykernel_12052\2332837819.py:4: SyntaxWarning: invalid escape sequence '\d'
  train['클리닉 내 총 시술 횟수'] = train['클리닉 내 총 시술 횟수'].str.extract('(\d+)').astype(int)


In [564]:
test['클리닉 내 총 시술 횟수']=test['클리닉 내 총 시술 횟수'].str.extract(r'(\d+)').astype(int)

In [565]:
train.groupby('IVF 시술 횟수')['임신 성공 여부'].mean()

IVF 시술 횟수
0회       0.286347
1회       0.250553
2회       0.245456
3회       0.235524
4회       0.227452
5회       0.212793
6회 이상    0.196082
Name: 임신 성공 여부, dtype: float64

In [566]:
# 어느 정도 선형적이 관계를 보이긴 함.. 숫자형으로만 바꾸기
train['IVF 시술 횟수']=train['IVF 시술 횟수'].str.extract(r'(\d+)').astype(int)
test['IVF 시술 횟수']=test['IVF 시술 횟수'].str.extract(r'(\d+)').astype(int)

In [567]:
train.groupby('DI 시술 횟수')['임신 성공 여부'].mean()

DI 시술 횟수
0회       0.260665
1회       0.191870
2회       0.217623
3회       0.242678
4회       0.219895
5회       0.188490
6회 이상    0.236801
Name: 임신 성공 여부, dtype: float64

In [568]:
# 다른 변수들로 대체 가능할지.. 알아보기 위해 일단 숫자형으로 바꾸기
train['DI 시술 횟수']=train['DI 시술 횟수'].str.extract(r'(\d+)').astype(int)
test['DI 시술 횟수']=test['DI 시술 횟수'].str.extract(r'(\d+)').astype(int)

In [569]:
train.groupby('DI 시술 횟수')['임신 성공 여부'].mean()

DI 시술 횟수
0    0.260665
1    0.191870
2    0.217623
3    0.242678
4    0.219895
5    0.188490
6    0.236801
Name: 임신 성공 여부, dtype: float64

In [570]:
num_cols=train.select_dtypes(exclude=['object']).columns
corr_values = train[num_cols].corr()['DI 시술 횟수']
# 0.9 이상 또는 -0.9 이하인 값만 필터링
corr_values[(corr_values >= 0.9) | (corr_values <= -0.9)]

DI 시술 횟수    1.0
Name: DI 시술 횟수, dtype: float64

딱히 대체 가능한 column은 없어 보임

일단 두기.. scaling 어떻게 할지.. 나중에 고민하기 ~

In [571]:
train['IVF 출산 횟수'].value_counts()

IVF 출산 횟수
0회    220759
1회     33347
2회      2021
3회       137
4회        13
5회         1
Name: count, dtype: int64

In [572]:
train.groupby('IVF 출산 횟수')['임신 성공 여부'].mean()

IVF 출산 횟수
0회    0.256574
1회    0.269889
2회    0.262741
3회    0.299270
4회    0.230769
5회    0.000000
Name: 임신 성공 여부, dtype: float64

In [573]:
test['IVF 출산 횟수'].value_counts()

IVF 출산 횟수
0회    77480
1회    11874
2회      666
3회       42
4회        5
Name: count, dtype: int64

In [574]:
train['IVF 출산 횟수']=train['IVF 출산 횟수'].str.extract(r'(\d+)').astype(int)
test['IVF 출산 횟수']=test['IVF 출산 횟수'].str.extract(r'(\d+)').astype(int)

In [575]:
# IVF 출산 횟수가 5회 인건 test 데이터에 없음 -> 지우기
train=train[train['IVF 출산 횟수']!=5]

In [576]:
train.groupby('IVF 출산 횟수')['임신 성공 여부'].mean()

IVF 출산 횟수
0    0.256574
1    0.269889
2    0.262741
3    0.299270
4    0.230769
Name: 임신 성공 여부, dtype: float64

In [577]:
num_cols=train.select_dtypes(exclude=['object']).columns
corr_values = train[num_cols].corr()['IVF 출산 횟수']
# 0.9 이상 또는 -0.9 이하인 값만 필터링
corr_values[(corr_values >= 0.9) | (corr_values <= -0.9)]

IVF 출산 횟수    1.0
Name: IVF 출산 횟수, dtype: float64

In [578]:
train.groupby('총 임신 횟수')['임신 성공 여부'].mean()

총 임신 횟수
0회       0.257482
1회       0.262922
2회       0.258008
3회       0.245638
4회       0.215054
5회       0.285714
6회 이상    0.333333
Name: 임신 성공 여부, dtype: float64

In [579]:
test.value_counts('총 임신 횟수')

총 임신 횟수
0회       72216
1회       15386
2회        2127
3회         297
4회          37
5회           3
6회 이상        1
Name: count, dtype: int64

In [580]:
train['총 임신 횟수']=train['총 임신 횟수'].str.extract(r'(\d+)').astype(int)
test['총 임신 횟수']=test['총 임신 횟수'].str.extract(r'(\d+)').astype(int)

In [581]:
train[['총 임신 횟수','임신 성공 여부']].corr()

,총 임신 횟수,임신 성공 여부
총 임신 횟수,1.000000,0.002723
임신 성공 여부,0.002723,1.000000


In [582]:
train['IVF 임신 횟수'].value_counts()

IVF 임신 횟수
0회       208220
1회        41518
2회         5788
3회          663
4회           81
5회            6
6회 이상         1
Name: count, dtype: int64

In [583]:
test['IVF 임신 횟수'].value_counts()

IVF 임신 횟수
0회    73207
1회    14571
2회     1981
3회      274
4회       31
5회        3
Name: count, dtype: int64

In [584]:
train['IVF 임신 횟수'] = train['IVF 임신 횟수'].str.extract('(\d+)').astype(int)
test['IVF 임신 횟수'] = test['IVF 임신 횟수'].str.extract('(\d+)').astype(int)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\sshyu\AppData\Local\Temp\ipykernel_12052\3354623911.py:1: SyntaxWarning: invalid escape sequence '\d'
  train['IVF 임신 횟수'] = train['IVF 임신 횟수'].str.extract('(\d+)').astype(int)
C:\Users\sshyu\AppData\Local\Temp\ipykernel_12052\3354623911.py:2: SyntaxWarning: invalid escape sequence '\d'
  test['IVF 임신 횟수'] = test['IVF 임신 횟수'].str.extract('(\d+)').astype(int)


In [585]:
# train에서만 임신 횟수가 6회 이상이 한개 나옴 -> 제거하기
train=train[train['IVF 임신 횟수']!=6]

In [586]:
train.groupby('IVF 임신 횟수')['임신 성공 여부'].mean()
# 5회에서 급격히 떨어지는 걸 볼 수 있음

IVF 임신 횟수
0    0.256882
1    0.266415
2    0.256047
3    0.247360
4    0.234568
5    0.166667
Name: 임신 성공 여부, dtype: float64

In [587]:
test['DI 임신 횟수'].value_counts()

DI 임신 횟수
0회    89008
1회      929
2회      117
3회       10
4회        2
5회        1
Name: count, dtype: int64

In [588]:
train['DI 임신 횟수'] = train['DI 임신 횟수'].str.extract('(\d+)').astype(int)
test['DI 임신 횟수'] = test['DI 임신 횟수'].str.extract('(\d+)').astype(int)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\sshyu\AppData\Local\Temp\ipykernel_12052\3173026808.py:1: SyntaxWarning: invalid escape sequence '\d'
  train['DI 임신 횟수'] = train['DI 임신 횟수'].str.extract('(\d+)').astype(int)
C:\Users\sshyu\AppData\Local\Temp\ipykernel_12052\3173026808.py:2: SyntaxWarning: invalid escape sequence '\d'
  test['DI 임신 횟수'] = test['DI 임신 횟수'].str.extract('(\d+)').astype(int)


In [589]:
train.groupby('DI 임신 횟수')['임신 성공 여부'].mean()
# 4회에서 임신 성공 여부가 모두 0이 되는 특이점..(row는 5개 존재함)

DI 임신 횟수
0    0.258870
1    0.209098
2    0.275956
3    0.212766
4    0.000000
5    0.333333
Name: 임신 성공 여부, dtype: float64

In [590]:
train.groupby('총 출산 횟수')['임신 성공 여부'].mean()
# 6회 이상 값이 0이 됨 (row 는 2개)
# test 에서 총 출산 횟수가 6회 이상 -> 임신 성공 여부를 0으로 나중에 고치기

총 출산 횟수
0회       0.256875
1회       0.267266
2회       0.263275
3회       0.275449
4회       0.230769
6회 이상    0.000000
Name: 임신 성공 여부, dtype: float64

In [591]:
test['총 출산 횟수'].value_counts()

총 출산 횟수
0회       76735
1회       12548
2회         727
3회          51
4회           5
6회 이상        1
Name: count, dtype: int64

In [592]:
train['총 출산 횟수'] = train['총 출산 횟수'].str.extract('(\d+)').astype(int)
test['총 출산 횟수'] = test['총 출산 횟수'].str.extract('(\d+)').astype(int)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\sshyu\AppData\Local\Temp\ipykernel_12052\1996552244.py:1: SyntaxWarning: invalid escape sequence '\d'
  train['총 출산 횟수'] = train['총 출산 횟수'].str.extract('(\d+)').astype(int)
C:\Users\sshyu\AppData\Local\Temp\ipykernel_12052\1996552244.py:2: SyntaxWarning: invalid escape sequence '\d'
  test['총 출산 횟수'] = test['총 출산 횟수'].str.extract('(\d+)').astype(int)


In [593]:
train.groupby('DI 출산 횟수')['임신 성공 여부'].mean()
# 5회 이면 0임. 따라서 마지막에 고치기

DI 출산 횟수
0회    0.258638
1회    0.229323
2회    0.236264
3회    0.222222
5회    0.000000
Name: 임신 성공 여부, dtype: float64

In [594]:
train['DI 출산 횟수'].value_counts()

DI 출산 횟수
0회    253946
1회      2128
2회       182
3회        18
5회         2
Name: count, dtype: int64

In [595]:
train['DI 출산 횟수'] = train['DI 출산 횟수'].str.extract('(\d+)').astype(int)
test['DI 출산 횟수'] = test['DI 출산 횟수'].str.extract('(\d+)').astype(int)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\sshyu\AppData\Local\Temp\ipykernel_12052\1227948566.py:1: SyntaxWarning: invalid escape sequence '\d'
  train['DI 출산 횟수'] = train['DI 출산 횟수'].str.extract('(\d+)').astype(int)
C:\Users\sshyu\AppData\Local\Temp\ipykernel_12052\1227948566.py:2: SyntaxWarning: invalid escape sequence '\d'
  test['DI 출산 횟수'] = test['DI 출산 횟수'].str.extract('(\d+)').astype(int)


In [596]:
train['난자 출처'].value_counts()

난자 출처
본인 제공     234289
기증 제공      15767
알 수 없음      6220
Name: count, dtype: int64

In [597]:
train.groupby('난자 출처')['임신 성공 여부'].mean()

난자 출처
기증 제공     0.315406
본인 제공     0.257985
알 수 없음    0.128457
Name: 임신 성공 여부, dtype: float64

In [598]:
test['난자 출처'].value_counts()

난자 출처
본인 제공     82397
기증 제공      5494
알 수 없음     2176
Name: count, dtype: int64

In [599]:
# 임신 성공 여부 평균을 반영하여 기증 제공이면 1, 본인 제공이면 0.5, 알수 없음이면 0으로
train['난자 기증 여부']=0.0
train.loc[train['난자 출처']=='기증 제공','난자 기증 여부']=1
train.loc[train['난자 출처']=='본인 제공','난자 기증 여부']=0.5

In [600]:
test['난자 기증 여부']=0.0
test.loc[test['난자 출처']=='기증 제공','난자 기증 여부']=1
test.loc[test['난자 출처']=='본인 제공','난자 기증 여부']=0.5

In [601]:
train.drop('난자 출처',axis=1,inplace=True)
test.drop('난자 출처',axis=1,inplace=True)

In [602]:
train.groupby('정자 출처')['임신 성공 여부'].mean()

정자 출처
기증 제공          0.243922
미할당            0.106557
배우자 및 기증 제공    0.000000
배우자 제공         0.260169
Name: 임신 성공 여부, dtype: float64

In [603]:
# 임신 성공 여부를 반영하여
# 기증 제공 또는 배우자 제공이면 1, 미할당이면 0.5 배우자 및 기증 제공이면 0
train['정자 기증 여부']=1.0
train.loc[train['정자 출처']=='미할당','정자 기증 여부']=0.5
train.loc[train['정자 출처']=='배우자 및 기증 제공','정자 기증 여부']=0

In [604]:
test['정자 기증 여부']=1.0
test.loc[test['정자 출처']=='미할당','정자 기증 여부']=0.5
test.loc[test['정자 출처']=='배우자 및 기증 제공','정자 기증 여부']=0

In [605]:
train.drop('정자 출처',axis=1,inplace=True)
test.drop('정자 출처',axis=1,inplace=True)

In [606]:
train['난자 기증자 나이'].value_counts()

난자 기증자 나이
알 수 없음     242307
만31-35세      6365
만26-30세      4976
만21-25세      2334
만20세 이하       294
Name: count, dtype: int64

94% 의 값이 알 수 없음임..(test 데이터도 마찬가지) 결측치를 채우면 오히려 성능이 떨어질 듯

In [607]:
test['정자 기증자 나이'].value_counts()

정자 기증자 나이
알 수 없음     81043
만21-25세     1988
만36-40세     1887
만26-30세     1828
만31-35세     1643
만41-45세     1299
만20세 이하      379
Name: count, dtype: int64

마찬가지로 90% 의 값이 알 수 없음임..(test 데이터도 마찬가지) 결측치를 채우면 오히려 성능이 떨어질 듯

In [608]:
train.drop(['난자 기증자 나이','정자 기증자 나이'],axis=1,inplace=True)
test.drop(['난자 기증자 나이','정자 기증자 나이'],axis=1,inplace=True)

In [609]:
train.select_dtypes(include=['object']).columns

Index(['시술 당시 나이', '특정 시술 유형'], dtype='object')

In [610]:
test.select_dtypes(include=['object']).columns

Index(['시술 당시 나이', '특정 시술 유형'], dtype='object')

In [611]:
train['배아 이식 경과일'].value_counts()

배아 이식 경과일
5.0    81459
3.0    57924
2.0    35078
0.0    24902
1.0     6053
4.0     4504
6.0     2773
7.0       90
Name: count, dtype: int64

원핫 인코딩

In [612]:
train['특정 시술 유형'].value_counts()
# 원핫 인코딩

특정 시술 유형
ICSI       128547
IVF         94568
Unknown     26960
IUI          6100
ICI            96
FER             3
GIFT            2
Name: count, dtype: int64

In [613]:
train=pd.get_dummies(train, columns=['특정 시술 유형'], drop_first=True,dtype=int)
test=pd.get_dummies(test, columns=['특정 시술 유형'], drop_first=True,dtype=int)

**아직 남아있는 NaN -> KNN imputation으로**

In [614]:
pd.DataFrame(train.isna().sum()[train.isna().sum() > 0])

,0
배아 이식 경과일,43493


In [615]:
pd.DataFrame(test.isna().sum()[test.isna().sum() > 0])

,0
배아 이식 경과일,15246


In [616]:
# 배아이식 경과일
num_cols=train.select_dtypes(exclude=['object']).columns
corr_values=train[num_cols].corr()['배아 이식 경과일']
corr_values[(corr_values >= 0.5) | (corr_values <= -0.5)]

배란 자극 여부            0.586438
총 생성 배아 수           0.648158
수집된 신선 난자 수         0.575872
혼합된 난자 수            0.625842
파트너 정자와 혼합된 난자 수    0.564743
동결 배아 사용 여부        -0.690088
신선 배아 사용 여부         0.691266
배아 이식 경과일           1.000000
특정 시술 유형_Unknown   -0.531194
Name: 배아 이식 경과일, dtype: float64

In [617]:
# 상관관계가 높은 feature들로 채워줌
from sklearn.impute import KNNImputer

# 사용할 컬럼 선택 
selected_columns = [
    '배란 자극 여부', '총 생성 배아 수', '수집된 신선 난자 수', '혼합된 난자 수', 
    '파트너 정자와 혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부', 
    '특정 시술 유형_Unknown'
]
# 선택한 컬럼만 포함한 데이터프레임 생성
train_selected = train[selected_columns]
# KNN Imputer 설정
imputer = KNNImputer(n_neighbors=5)  # K값은 5로 설정 
train_selected_imputed = pd.DataFrame(imputer.fit_transform(train_selected), columns=selected_columns)
# 원본 데이터프레임에 채워진 값 반영
train[selected_columns] = train_selected_imputed
# 이제 '배아 이식 경과일'만 KNN으로 채우기
train['배아 이식 경과일'] = train['배아 이식 경과일'].fillna(train['배아 이식 경과일'].mean())

In [618]:
train['배아 이식 경과일'].value_counts()

배아 이식 경과일
5.0    84202
3.0    59286
2.0    36852
0.0    24903
1.0     8363
0.6     6058
4.0     5907
2.2     3757
4.6     3434
2.4     3385
0.8     2952
6.0     2773
2.6     2445
4.2     2082
2.8     1533
3.8     1514
3.6     1264
4.4     1003
0.4      900
1.8      748
3.2      685
3.4      674
4.8      550
5.2      485
1.6      247
5.4       90
7.0       90
1.4       15
5.6        4
Name: count, dtype: int64

In [619]:
# 선택한 컬럼만 포함한 데이터프레임 생성
test_selected = test[selected_columns]

# KNN Imputer 설정
imputer = KNNImputer(n_neighbors=5)  # K값은 5로 설정 
test_selected_imputed = pd.DataFrame(imputer.fit_transform(test_selected), columns=selected_columns)

# 원본 데이터프레임에 채워진 값 반영
test['배아 이식 경과일'] = test_selected_imputed['배아 이식 경과일']

In [620]:
test['배아 이식 경과일'].value_counts()

배아 이식 경과일
5.0    29861
3.0    20898
2.0    12588
0.0    10173
1.0     2094
4.0     1995
2.2     1652
0.6     1584
2.4     1399
4.6     1079
0.4     1015
6.0      960
4.2      730
2.6      676
2.8      600
3.4      467
3.8      432
3.2      385
4.4      376
3.6      343
4.8      205
5.2      148
1.8      122
1.6      104
1.4       98
5.4       35
7.0       33
0.8        6
1.2        4
5.8        3
5.6        2
Name: count, dtype: int64

In [646]:
train['배아 이식 경과일'].isna().sum()

75

In [621]:
train['시술 당시 나이'].value_counts()

시술 당시 나이
만18-34세    102438
만35-37세     57765
만38-39세     39240
만40-42세     37340
만43-44세     12249
만45-50세      6915
알 수 없음        329
Name: count, dtype: int64

In [622]:
test['시술 당시 나이'].value_counts()

시술 당시 나이
만18-34세    36178
만35-37세    20211
만38-39세    13611
만40-42세    13266
만43-44세     4272
만45-50세     2413
알 수 없음       116
Name: count, dtype: int64

In [ ]:
train['시술 나이'] = np.nan
age_mapping = {
    '만18-34세': 1
    '만35-37세': 2,
    '만38-39세': 3,
    '만40-42세': 4,
    '만43-44세': 5,
    '만45-50세': 6
}
train['시술 나이'] = train['시술 당시 나이'].map(age_mapping)
train.loc[train['시술 당시 나이'] == '알 수 없음', '시술 나이'] = np.nan
train['시술 나이'] = train['시술 나이'].astype(float)

In [634]:
train['시술 나이'].isna().sum()

329

In [638]:
train[train['시술 당시 나이']=='알 수 없음'].shape[0]

329

In [637]:
test['시술 나이'] = np.nan
test['시술 나이'] = test['시술 당시 나이'].map(age_mapping)
test.loc[test['시술 당시 나이'] == '알 수 없음', '시술 나이'] = np.nan
test['시술 나이'] = test['시술 나이'].astype(float)

In [639]:
test['시술 나이'].isna().sum()

116

In [640]:
test[test['시술 당시 나이']=='알 수 없음'].shape[0]

116

In [644]:
test['시술 나이'].value_counts()

시술 나이
1.0    36178
2.0    20211
3.0    13611
4.0    13266
5.0     4272
6.0     2413
Name: count, dtype: int64

In [625]:
test['시술 나이']=0
test.loc[test['시술 당시 나이']=='만35-37세','시술 나이']=1
test.loc[test['시술 당시 나이']=='만38-39세','시술 나이']=2
test.loc[test['시술 당시 나이']=='만40-42세','시술 나이']=3
test.loc[test['시술 당시 나이']=='만43-44세','시술 나이']=4
test.loc[test['시술 당시 나이']=='만45-50세','시술 나이']=5
test['시술 나이'] = test['시술 나이'].astype(float)
test.loc[test['시술 나이'] == '알 수 없음', '시술 나이'] = np.nan

In [627]:
train['시술 나이'].isna().sum()

0

In [498]:
# train.drop('시술 당시 나이',inplace=True, axis=1)
# test.drop('시술 당시 나이',inplace=True, axis=1)

In [503]:
num_cols=train.select_dtypes(exclude=['object']).columns
corr_values=train[num_cols].corr()['시술 나이']

In [507]:
# 위에 비해 값이 correlation 값 자체가 낮음 따라서 0.2를 기준으로
corr_values[(corr_values >= 0.2) | (corr_values <= -0.2)]

배란 자극 여부           -0.207645
총 시술 횟수             0.217266
IVF 시술 횟수           0.216393
수집된 신선 난자 수        -0.307762
혼합된 난자 수           -0.223943
파트너 정자와 혼합된 난자 수   -0.222229
난자 기증 여부            0.206043
시술 나이               1.000000
Name: 시술 나이, dtype: float64

In [511]:
train['시술 나이'].value_counts()

시술 나이
0.0    102767
1.0     57765
2.0     39240
3.0     37340
4.0     12249
5.0      6915
Name: count, dtype: int64

In [513]:
train['시술 나이'].isna().sum()

0